# Stacks and Queues (Lines)

## Agenda

1. Stacks
     - ... for delimiter pairing
     - ... for postfix expression evaluation
     - ... for tracking execution and *backtracking*
2. Queues
    - ... for tracking execution and *backtracking*
    - ... for apportioning work
    - ... for fair scheduling (aka "round-robin" scheduling)
3. Run-time analysis
4. Closing remarks

## Overview

While the list data structure is incredibly useful, both implementations we explored (array-backed and linked) have operations that run in $O(N)$ time, which make them non-ideal for use with large, growing collections of data.

By further restricting the list API, however — in particular, by *isolating points of access to either the front or end of the data set* — we can create data structures whose operations are all $O(1)$, and remain very useful in their own right.

## 1. Stacks

Stacks are linear data structures which only permit access to one "end" of the data collection. We can only append ("push") items onto the tail end (a.k.a. the "top") of a stack, and only the most recently added item can be removed ("popped"). The last item to be pushed onto a stack is therefore the first one to be popped off, which is why we refer to stacks as last-in, first out (LIFO) structures.

In [1]:
# array-backed implementation

class Stack:
    def __init__(self):
        self.data = []
        
    def push(self, val): #Adding something on to a stack
        self.data.append(val) #O(1)
        
    
    def pop(self): #O(1)
        assert(self.data)
        r = self.data[-1]
        del self.data[-1]
        return r
    
    def empty(self): #O(1)
        return len(self.data) == 0

In [2]:
s = Stack()
for x in range(10):
    s.push(x)

In [3]:
s.pop()

9

In [4]:
# linked implementation

class Stack:
    class Node:#Linked version so that we never have to recopy the array due to memory issues, useful for dynamic stacks
        def __init__(self, val, next=None):
            self.val = val
            self.next  = next
    
    def __init__(self):
        self.top = None

    def push(self, val): #O(1)
        self.top = Stack.Node(val, next = self.top)
        
    def pop(self): #O(1)
        assert(self.top), 'Stack is empty'
        r = self.top.val
        self.top = self.top.next
        return r
    
    def empty(self): #O(1)
        return self.top is None

In [5]:
s = Stack()
for x in range(10):
    s.push(x)

In [6]:
s.pop()

9

### ... for delimiter pairing

e.g., `'(1 + 2 * (3 - (4 / 2) + 5) - (6 + 1))'`

In [7]:
def check_parens(expr):
    s = Stack()
    for c in expr:
        if c =='(':
            s.push(c)
        elif c == ')':
            try:
                s.pop()
            except:
                return False
    return s.empty()

In [8]:
check_parens('()')

True

In [9]:
check_parens('((()))')

True

In [10]:
check_parens('()(()()(()))')

True

In [11]:
check_parens('(')

False

In [12]:
check_parens('())')

False

In [13]:
check_parens('(1 + 2 * (3 - (4 / 2) + 5) - (6 + 1))')

True

### ... for postfix expression (aka "reverse polish notation") evaluation

e.g., `'(1 + 2) * 5'` $\rightarrow$ `'1 2 + 5 *'`

In [14]:
def eval_postfix(expr):
    s = Stack()
    toks = expr.split()
    for t in toks:
        if t.isdigit():
            s.push(int(t))
        elif t == '+':
            s.push(s.pop() + s.pop())
        elif t == '*':
            s.push(s.pop()*s.pop())
    return s.pop()

In [15]:
eval_postfix('1 2 + 5 *')

15

In [16]:
eval_postfix('1 2 5 * +')

11

In [17]:
# ((1 + 2) * (3 + 2)) * 10
eval_postfix('1 2 + 3 2 + * 10 *')

150

### ... for tracking execution and backtracking

In [18]:
maze_str = """######   
              I    #   
              # ## #   
              # ####   
              #    O   
              ######"""

def parse_maze(maze_str):
    grid = []
    for line in maze_str.split('\n'):
        grid.append(['# IO'.index(c) for c in line.strip()])
    return grid

def print_maze(grid):
    for r in grid:
        print(''.join('# IO!+'[c] for c in r))

In [19]:
parse_maze(maze_str)

[[0, 0, 0, 0, 0, 0],
 [2, 1, 1, 1, 1, 0],
 [0, 1, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 3],
 [0, 0, 0, 0, 0, 0]]

In [20]:
maze = parse_maze(maze_str)
maze[1][1] = 4
print_maze(maze)

######
I!   #
# ## #
# ####
#    O
######


In [21]:
class Move:
    def __init__(self, frm, to):
        self.frm = frm
        self.to  = to
        
    def __repr__(self):
        return '({},{}) -> ({},{})'.format(self.frm[0], self.frm[1],
                                           self.to[0],  self.to[1])

def moves(maze, place):
    moves = [Move(place, (place[0]+d[0], place[1]+d[1]))
            for d in ((-1, 0), (1, 0), (0, -1), (0, 1))
            if place[0]+d[0] in range(len(maze)) and 
               place[1]+d[1] in range(len(maze[0])) and
               maze[place[0]+d[0]][place[1]+d[1]] in (1, 2, 3)]
    return moves

In [22]:
from time import sleep
from IPython.display import clear_output

def visit(maze, loc):
    """Annotates a loc in the maze as visited"""
    maze[loc[0]][loc[1]] = 5
    
def mark(maze, loc):
    """Annotates a loc in the maze as being of interest"""
    if maze[loc[0]][loc[1]] != 3:
        maze[loc[0]][loc[1]] = 4
    
def display(maze):
    clear_output(True)
    print_maze(maze)
    sleep(0.5)

def solve_maze(maze, entry):
    for m in moves(maze, entry):
        save_move(m)
    visit(maze, entry)
    while not out_of_moves():#
        move = next_move()#
        if maze[move.to[0]][move.to[1]] == 3:
            return True
        display(maze)
        visit(maze, move.to)
        for m in moves(maze, move.to):
            mark(maze, m.to)
            save_move(m)
    return False

In [23]:
move_stack = Stack()

def save_move(move):
    move_stack.push(move)

def next_move():
    return move_stack.pop()

def out_of_moves():
    return move_stack.empty()

In [24]:
solve_maze(parse_maze(maze_str), (1, 0))

######
+++++#
#+##+#
#+####
#+++!O
######


True

In [25]:
maze_str = """#################
              I #       #     #
              # ##### # # # # #
              #     # # # # # #
              # ### ### # # ###
              #   #       #   O
              #################"""

In [26]:
solve_maze(parse_maze(maze_str), (1, 0))

#################
++#       #+++++#
#+##### # #+#+#+#
#+++++# # #+#+#+#
#!###+###!#+#+###
#   #+++++++#++!O
#################


True

In [27]:
maze_str = """#################
              I               #
              # # # # # # # # #
              # # # # # # # # #
              # ###############
              #               O
              #################"""

In [28]:
solve_maze(parse_maze(maze_str), (1, 0))

#################
++++++++++++++++#
#+#+#+#+#+#+#+#+#
#+#+#+#+#+#+#+#+#
#+###############
#++++++++++++++!O
#################


True

## 2. Queues

Queues are linear data structures wherein we are only permitted to append ("enqueue") items onto the rear, and remove ("dequeue") items from the front. The oldest item still in a queue is therefore the next one to be dequeued, which is why we refer to a queue as a first-in, first-out (FIFO) structure. It is helpful to think of a queue as being the model for a line at a typical supermarket checkout aisle (first customer in, first customer to be checked out).

In [29]:
# array-backed implementation

class Queue:
    def __init__(self):
        self.hidx = -1
        self.data = []

    def enqueue(self, val):
        self.data.append(val) # O(1)
        
    def dequeue(self): # O(n)
        assert(not self.empty())
        self.hidx += 1
        r = self.data[self.hidx]
#         del self.data[0]
        self.data[self.hidx] = None
        return r
    
    def empty(self):
        return self.hidx == len(self.data)-1

In [30]:
q = Queue()
for x in range(10):
    q.enqueue(x)

In [31]:
q.dequeue()

0

In [32]:
# linked implementation

class Queue:
    class Node:
        def __init__(self, val, next=None):
            self.val = val
            self.next  = next
    
    def __init__(self):
        self.head = self.tail = None

    def enqueue(self, val):
        if not self.empty():
            self.tail.next = self.tail = Queue.Node(val)
        else:
            self.head = self.tail = Queue.Node(val)
    def dequeue(self):
        assert(self.head)
        r = self.head.val
        self.head = self.head.next
        if not self.head:
            self.tail = None
        return r
    
    def empty(self):
        return self.head == None
    
    def __iter__(self):
        n = self.head
        while n:
            yield n.val
            n = n.next
    
    def __repr__(self): #No purpose, not supposed to look at middle of queue
        return '<' + ','.join(str(x) for x in self) + '>'

In [33]:
q = Queue()
for x in range(10):
    q.enqueue(x)

In [34]:
for x in range(5):
    q.enqueue(x)

In [35]:
q

<0,1,2,3,4,5,6,7,8,9,0,1,2,3,4>

In [36]:
for i in q:
    print(i)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4


In [37]:
q.dequeue()

0

### ... for tracking execution and backtracking

In [38]:
move_queue = Queue()

def save_move(move):
    move_queue.enqueue(move)

def next_move():
    move_queue.dequeue()

def out_of_moves():
    move_queue.empty

In [39]:
maze_str = """######   
              I    #   
              # ## #   
              # ####   
              #    O   
              ######"""

In [40]:
solve_maze(parse_maze(maze_str), (1, 0))

AttributeError: 'NoneType' object has no attribute 'to'

In [ ]:
maze_str = """#################
              I #       #     #
              # ##### # # # # #
              #     # # # # # #
              # ### ### # # ###
              #   #       #   O
              #################"""

In [ ]:
solve_maze(parse_maze(maze_str), (1, 0))

In [ ]:
maze_str = """#################
              I               #
              # # # # # # # # #
              # # # # # # # # #
              # ###############
              #               O
              #################"""

In [ ]:
solve_maze(parse_maze(maze_str), (1, 0))

### ... for doling out work

In [58]:
from threading import Thread, Lock
from time import sleep
import random

lock = Lock() #Prevents concurrency errors
def worker_fn(cid, q):
    while True:
        try:
            with lock:
                work = q.dequeue()
        except: # queue is empty
            sleep(1)
            continue
        if work == 'Stop':
            print('Consumer', cid, 'stopping.')
            break
        else:
            print('Consumer', cid, 'processing', work)
            sleep(random.random())

work_queue = Queue()
for i in range(5):
    Thread(target=worker_fn, args=(i, work_queue)).start()

In [59]:
import threading
threading.active_count()

15

In [60]:
for i in range(10):
    with lock:
        work_queue.enqueue(i)

ConsumerConsumer 3 processing 1
 0 processing 0
Consumer 4 processing 2
Consumer 2 processing 3
Consumer 1 processing 4
Consumer 0 processing 5
Consumer 1 processing 6
Consumer 4 processing 7
Consumer 3 processing 8


In [61]:
for i in range(5):
    with lock:
        work_queue.enqueue('Stop')

Consumer 0 processing 9
Consumer 0 stopping.
Consumer 4 stopping.
Consumer 1 stopping.
Consumer 2 stopping.
Consumer 3 stopping.


### ... for fair scheduling (aka "round-robin" scheduling)

In [62]:
from random import randint
from time import sleep

task_queue = Queue()
for i in range(3):
    task_queue.enqueue(('Job {}'.format(i), randint(3, 6)))

In [63]:
n = task_queue.head
while n:
    print(n.val)
    n = n.next

('Job 0', 4)
('Job 1', 3)
('Job 2', 4)


In [64]:
while not task_queue.empty():
    job, time_left = task_queue.dequeue()
    print('Running', job)
    sleep(1)
    time_left -= 1
    if time_left > 0:
        print('Re-queueuing', job, 'with remaining time =', time_left)
        task_queue.enqueue((job, time_left))
    else:
        print('*', job, 'done')

Running Job 0
Re-queueuing Job 0 with remaining time = 3
Running Job 1
Re-queueuing Job 1 with remaining time = 2
Running Job 2
Re-queueuing Job 2 with remaining time = 3
Running Job 0
Re-queueuing Job 0 with remaining time = 2
Running Job 1
Re-queueuing Job 1 with remaining time = 1
Running Job 2
Re-queueuing Job 2 with remaining time = 2
Running Job 0
Re-queueuing Job 0 with remaining time = 1
Running Job 1
* Job 1 done
Running Job 2
Re-queueuing Job 2 with remaining time = 1
Running Job 0
* Job 0 done
Running Job 2
* Job 2 done


## 3. Run-time analysis

Stack & Queue implementations:

- Insertion (push and enqueue) = $O(1)$
- Deletion (pop and dequeue) = $O(1)$

## 4. Closing remarks